In [1]:
import sys
import os
import yaml
import numpy 
import pandas
import pathlib
import time
from io import StringIO
from datetime import date
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.offsetbox import AnchoredText
from matplotlib.ticker import MaxNLocator
import matplotlib as mpl

# Create dataframe with nitrogen loadings

In [2]:
def Table1_loadings(case):

    # open yaml configuration file created by notebook `SSM_config_{case}.ipynb` 
    with open(f'../../etc/SSM_config_{case}.yaml', 'r') as file:
        ssm = yaml.safe_load(file)
    # load list of directory paths for SSM nutrient loading .dat files
    runs = [*ssm['paths']['nutrient_loading_inputs']]
    # print out file list
    for run in runs:
        print(ssm['paths']['nutrient_loading_inputs'][run])
    # establish directory path for ssm_pnt_wq_station_info.xlsx
    in_dir = pathlib.Path('/mmfs1/gscratch/ssmc/USRS/PSI/Rachael/projects/KingCounty/SalishSeaModel-spreadsheets')

    ### START: move to util script ###
    
    # Adaptation of Ben's method for processing output    
    inputs = {}
    for run in runs:
        with open(ssm['paths']['nutrient_loading_inputs'][run]) as f:
            # The parsing logic here is is derived from the linkage instructions for the
            # model and direct examination of the source code

            # The first line does not contain important information and is treated only like
            # a filetype magic
            next(f)

            # The total number of discharge nodes
            num_qs = int(next(f))
            # All the node numbers with discharges
            #nodes = np.loadtxt([next(f) for l in range(num_qs)], comments='!', dtype=int)
            node_raw = StringIO('\n'.join([next(f) for l in range(num_qs)]))
            node_df = pandas.read_csv(node_raw, sep='\s+!\s+', names=('Node','Comment'),
                                  dtype={'Node':numpy.int64,'Comment':object}, engine='python')
            node_df.set_index('Node', inplace=True)
            nodes = node_df.index.to_numpy()
            # Depth distribution fractions into each node. Skipping the first (node count) column
            vqdist = numpy.loadtxt([next(f) for l in range(num_qs)])[:,1:]

            num_times = int(next(f))

            # Initialize storage arrays
            times = numpy.zeros(num_times)
            qs = numpy.zeros((num_times, num_qs))
            # State variables in the order they are present in the file. These are also going
            # to be the NetCDF variable names
            statevars = ('discharge', 'temp', 'salt', 'tss',  'alg1', 'alg2', 'alg3', 'zoo1',
                                      'zoo2', 'ldoc', 'rdoc', 'lpoc', 'rpoc', 'nh4',  'no32',
                                      'urea', 'ldon', 'rdon', 'lpon', 'rpon', 'po4',  'ldop',
                                      'rdop', 'lpop', 'rpop', 'pip',  'cod',  'doxg', 'psi',
                                      'dsi',  'alg1p','alg2p','alg3p','dic',  'talk')
            inputs[run] = {}
            for v in statevars:
                inputs[run][v] = numpy.zeros((num_times, num_qs))

            for t in range(num_times):
                times[t] = float(next(f))
                for v in statevars:
                    inputs[run][v][t,:] = numpy.loadtxt([next(f)])
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # Create dictionaries of loading and discharge information by run
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    loading={}
    total_flow={}
    total_nitrogen={}
    #total_annual_nitrogen={}
    for run in runs:
        print(run)
        loading[run] = (inputs[run]['discharge'] * (inputs[run]['nh4'] + inputs[run]['no32']) * 24 * 3600)/1000 #m3/s*mg/l -> kg/day
        total_flow[run] = (inputs[run]['discharge'] * 24 * 3600).sum(axis=0) # annual discharge m3/year
        total_nitrogen[run]=loading[run].sum(axis=0) # annual loading over 366 days (g/year)
        #total_annual_nitrogen[run] = total_nitrogen[run].sum() # g/year -> kg/year
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # import source location names from excel
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # information in excel spreadsheet: type (" River" or " Point Source"), node_id, layer_distribute, basin, country, NH4[kg/year], NO3NO2[kg/year], N-load
    source_locations = pandas.read_excel(in_dir/'ssm_pnt_wq_station_info.xlsx',index_col='Unnamed: 0')
    river_locations = source_locations[source_locations['type']==' River']
    wwtp_locations = source_locations[source_locations['type']==' Point Source']
    # make lists of names for: 
    all_rivers_list = river_locations.index.to_list() # river input locations
    all_wwtps_list = wwtp_locations.index.to_list()   # WWTP input locations
    all_locations_list = source_locations.index.to_list()  # both river and WWTP input locations
    all_locations_type_list = source_locations['type'].to_list() # type of input (" River" or " Point Source") to eliminate duplicate names
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # Create dataframes of information using names from excel as indices
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # NOTE: Some names are redundant for rivers and WWTP so we need to use both name and type (" River" vs. " Point Source")
    total_flow_df  = pandas.DataFrame(
        total_flow, 
        index=all_locations_list,
        columns=ssm['run_information']['run_tag'][case] # Assign report names to columns
    )
    total_loading_df = pandas.DataFrame(
        total_nitrogen, 
        index=all_locations_list,
        columns=ssm['run_information']['run_tag'][case] # Assign report names to columns
    )
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # Use both name of source and type to create lists of wwtp and river sources
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # Create boolean arrays with True for only WWTP and River sources used in this cases 
    case_wwtp = numpy.asarray(total_loading_df.index.isin(ssm['WWTP_names']) & (source_locations['type']==' Point Source'))
    case_river = numpy.asarray(total_loading_df.index.isin(ssm['river_names']) & (source_locations['type']==' River'))
    # Create boolean arrays with True for all WWTP and River sources used in this study
    all_wwtp= numpy.asarray(total_loading_df.index.isin(all_wwtps_list) & (source_locations['type']==' Point Source'))
    all_river = numpy.asarray(total_loading_df.index.isin(all_rivers_list) & (source_locations['type']==' River'))
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # Calculate the total nitrogen loading for rivers and wwtps
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # modified in a particular case
    total_scenario_wwtp_nitrogen_df = total_loading_df[case_wwtp]
    total_scenario_river_nitrogen_df = total_loading_df[case_river]
    # included in SSM
    total_wwtp_nitrogen_df = total_loading_df[all_wwtp]
    total_river_nitrogen_df = total_loading_df[all_river]    
    # total nitrogen loads, by source and scenario
    wwtp_loads = total_loading_df[case_wwtp]
    river_loads = total_loading_df[case_river]
    # total nitrogen loads across local and all sources, by scenario
    # rename columns in the process
    total_wwtp_loading_local={}
    total_river_loading_local={}
    total_wwtp_loading_all={}
    total_river_loading_all={}
    for scenario in [*total_scenario_wwtp_nitrogen_df]:
        total_wwtp_loading_local[ssm['run_information']['run_tag'][case][scenario]]=total_scenario_wwtp_nitrogen_df[scenario].sum()
        total_river_loading_local[ssm['run_information']['run_tag'][case][scenario]]=total_scenario_river_nitrogen_df[scenario].sum()
        total_wwtp_loading_all[ssm['run_information']['run_tag'][case][scenario]]=total_wwtp_nitrogen_df[scenario].sum()
        total_river_loading_all[ssm['run_information']['run_tag'][case][scenario]]=total_river_nitrogen_df[scenario].sum()
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # Creating dataframe with loading values
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    print('Creating WWTP and River loadings dataframe')
    wwtp_load_df = pandas.DataFrame(wwtp_loads).sort_values(by=['wqm_baseline'], ascending=False)
    river_load_df = pandas.DataFrame(river_loads).sort_values(by=['wqm_baseline'], ascending=False)
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # Rename columns for report
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    wwtp_load_df=wwtp_load_df.rename(columns=ssm['run_information']['run_tag'][case])
    river_load_df=river_load_df.rename(columns=ssm['run_information']['run_tag'][case])
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # Creating dataframe with discharge values
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    wwtp_flow_df = pandas.DataFrame({'Annual Total Flow (m^3/year)':total_flow['wqm_baseline'][case_wwtp]}, index=ssm['WWTP_names'])
    river_flow_df = pandas.DataFrame({'Annual Total Flow (m^3/year)':total_flow['wqm_baseline'][case_river]}, index=ssm['river_names'])
    # combine river discharge and loading in locations where there are duplicates (R_1, R_2)
    flow = []
    river_load = {}
    # River discharge (not scenario-dependant)
    for river in ssm['unique_river_names']:      
        flow.append(river_flow_df[river_flow_df.index.str.contains(river)]["Annual Total Flow (m^3/year)"].sum())
    # River loading (scenario dependant)
    for scenario in [*river_load_df]:
        river_load[scenario]=[]
        for river in ssm['unique_river_names']: 
            river_load[scenario].append(river_load_df[river_load_df.index.str.contains(river)][scenario].sum())
    # dictionary of river discharge with one value per river instead of two (R_1 and R_2)
    river_flow_df_corrected = pandas.DataFrame({'Annual Total Flow (m^3/year)':flow},index=ssm['unique_river_names'])
    # dictionary of river loadings with one value per river instead of two (R_1 and R_2)
    river_load_df_corrected = pandas.DataFrame(river_load,index=ssm['unique_river_names'])    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # Combine dataframes with loading and discharge
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ 
    # RIVERS
    river_load_df_corrected["Annual Total Flow (m^3/year)"] = river_flow_df_corrected["Annual Total Flow (m^3/year)"]
    # WWTP
    wwtp_load_df["Annual Total Flow (m^3/year)"] = wwtp_flow_df["Annual Total Flow (m^3/year)"]
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # Create dataframes for total loadings (a) used in this study, and (b) used in SSM
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ 
    # RIVERS
    total_river_loading_local_df = pandas.DataFrame(total_river_loading_local, index=['Total Rivers (altered in this report)'])
    total_river_loading_all_df = pandas.DataFrame(total_river_loading_all, index=['Total Rivers (all in model domain) '])
    # WWTP
    total_wwtp_loading_local_df = pandas.DataFrame(total_wwtp_loading_local, index=['Total WWTPs (altered in this report)'])
    total_wwtp_loading_all_df = pandas.DataFrame(total_wwtp_loading_all, index=['Total WWTPs (all in model domain) '])
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # Combine dataframes with loadings/discharge with those showing total loadings in this study and all in SSM
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # RIVERS
    river_load_df_corrected=river_load_df_corrected.append(total_river_loading_local_df)
    river_load_df_corrected=river_load_df_corrected.append(total_river_loading_all_df)
    # WWTP
    wwtp_load_df=wwtp_load_df.append(total_wwtp_loading_local_df)
    wwtp_load_df=wwtp_load_df.append(total_wwtp_loading_all_df)

    
   
    # make README 
    this_file = '=HYPERLINK("https://github.com/UWModeling/KingCounty-Rachael/blob/main/notebooks/reports/Table1_NutrientLoadings.ipynb")'
    run_description = '=HYPERLINK("https://github.com/RachaelDMueller/KingCounty-Rachael/blob/main/docs/supporting/KingCounty_Model_Runs.xlsx","KingCounty_Model_Runs.xlsx (USING ORIGINAL RUN TAGS!!!)")'

    created_by = 'Rachael D. Mueller'
    created_at = 'Puget Sound Institute'
    created_from = 'Model results produced by Su Kyong Yun (PNNL) and Rachael Mueller (PSI)'
    units='kg/year'
    created_on = date.today().strftime("%B %d, %Y")
    contact = 'Rachael D Mueller (rdmseas@uw.edu)'
    header = {
        ' ':[created_by, created_at, created_on, this_file, 
            units, contact, created_from, 
            run_description]
    }
    header_df = pandas.DataFrame(header, index=[
        'Created by',
        'Created at',                           
        'Created on',
        'Created with',
        'Loading units',
        'Contacts',
        'Modeling by',
        'Model Run Overview'])

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # Save output to excel
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    excel_output_path = pathlib.Path('/mmfs1/gscratch/ssmc/USRS/PSI/Rachael/projects/KingCounty/reports')
    print('*************************************************************')
    print('Writing spreadsheet to: ',excel_output_path)
    print('*************************************************************')
    if os.path.exists(excel_output_path)==False:
        print(f'creating: {excel_output_path}')
        os.umask(0) #clears permissions
        os.makedirs(excel_output_path, mode=0o777,exist_ok=True)
    with pandas.ExcelWriter(
        excel_output_path/f'Table1_NutrientLoadings_{case}.xlsx', mode='w') as writer:  
        wwtp_load_df.to_excel(writer, sheet_name=f'WWTP ({case})')
        river_load_df_corrected.to_excel(writer, sheet_name=f'Rivers ({case})')
        header_df.to_excel(writer, sheet_name='README')
    print(f'Number of WWTPs in this case: {len(total_scenario_wwtp_nitrogen_df)}')
    print(f'Number of rivers in this case: {len(total_scenario_river_nitrogen_df)}')
    print(f'Number of WWTPs in model: {len(total_wwtp_nitrogen_df)}')
    print(f'Number of rivers in model: {len(total_river_nitrogen_df)}')
    
    return total_wwtp_nitrogen_df, total_river_nitrogen_df#wwtp_load_df, river_load_df_corrected

In [3]:
wwtp_load_df, river_load_df = Table1_loadings('main')

/mmfs1/gscratch/ssmc/USRS/PSI/Rachael/projects/KingCounty/SalishSeaModel/4b/hotstart/inputs/ssm_pnt_wq.dat
/mmfs1/gscratch/ssmc/USRS/PSI/Rachael/projects/KingCounty/SalishSeaModel/4c/hotstart/inputs/ssm_pnt_wq.dat
/mmfs1/gscratch/ssmc/USRS/PSI/Sukyong/kingcounty/4x_inputs/ssm_pnt_wq_4c.dat
/mmfs1/gscratch/ssmc/USRS/PSI/Sukyong/kingcounty/4x_inputs/ssm_pnt_wq_4d.dat
/mmfs1/gscratch/ssmc/USRS/PSI/Sukyong/kingcounty/4x_inputs/ssm_pnt_wq_4e.dat
/mmfs1/gscratch/ssmc/USRS/PSI/Sukyong/kingcounty/4x_inputs/ssm_pnt_wq_4f.dat
/mmfs1/gscratch/ssmc/USRS/PSI/Sukyong/kingcounty/4x_inputs/ssm_pnt_wq_4g.dat
/mmfs1/gscratch/ssmc/USRS/PSI/Sukyong/kingcounty/4x_inputs/ssm_pnt_wq_4h.dat
/mmfs1/gscratch/ssmc/USRS/PSI/Sukyong/kingcounty/4x_inputs/ssm_pnt_wq_4i.dat
/mmfs1/gscratch/ssmc/USRS/PSI/Sukyong/kingcounty/4x_inputs/ssm_pnt_wq_4j.dat
/mmfs1/gscratch/ssmc/USRS/PSI/Rachael/projects/KingCounty/SalishSeaModel/4k/hotstart/inputs/ssm_pnt_wq.dat
/mmfs1/gscratch/ssmc/USRS/PSI/Sukyong/kingcounty/4x_inputs/ssm_

/tmp/ipykernel_23434/1666772707.py:184: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  river_load_df_corrected=river_load_df_corrected.append(total_river_loading_local_df)
/tmp/ipykernel_23434/1666772707.py:185: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  river_load_df_corrected=river_load_df_corrected.append(total_river_loading_all_df)
/tmp/ipykernel_23434/1666772707.py:187: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  wwtp_load_df=wwtp_load_df.append(total_wwtp_loading_local_df)
/tmp/ipykernel_23434/1666772707.py:188: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  wwtp_load_df=wwtp_load_df.append(total_wwtp_loading_all_df)


*************************************************************
Writing spreadsheet to:  /mmfs1/gscratch/ssmc/USRS/PSI/Rachael/projects/KingCounty/reports
*************************************************************
Number of WWTPs in this case: 3
Number of rivers in this case: 2
Number of WWTPs in model: 99
Number of rivers in model: 160


In [2]:
case = 'whidbey'
# open yaml configuration file created by notebook `SSM_config_{case}.ipynb` 
with open(f'../../etc/SSM_config_{case}.yaml', 'r') as file:
    ssm = yaml.safe_load(file)
# load list of directory paths for SSM nutrient loading .dat files
runs = [*ssm['paths']['nutrient_loading_inputs']]
# print out file list
for run in runs:
    print(ssm['paths']['nutrient_loading_inputs'][run])
# establish directory path for ssm_pnt_wq_station_info.xlsx
in_dir = pathlib.Path('/mmfs1/gscratch/ssmc/USRS/PSI/Rachael/projects/KingCounty/SalishSeaModel-spreadsheets')


/mmfs1/gscratch/ssmc/USRS/PSI/Sukyong/kingcounty/WQM/WQM/hotstart/inputs/ssm_pnt_wq.dat
/mmfs1/gscratch/ssmc/USRS/PSI/Sukyong/kingcounty/WQM_REF/WQM_REF/hotstart/inputs/ssm_pnt_wq.dat
/mmfs1/gscratch/ssmc/USRS/PSI/Sukyong/kingcounty/3b/inputs/ssm_pnt_wq.dat
/mmfs1/gscratch/ssmc/USRS/PSI/Sukyong/kingcounty/3e/inputs/ssm_pnt_wq.dat
/mmfs1/gscratch/ssmc/USRS/PSI/Sukyong/kingcounty/3f/inputs/ssm_pnt_wq.dat
/mmfs1/gscratch/ssmc/USRS/PSI/Sukyong/kingcounty/3g/inputs/ssm_pnt_wq.dat
/mmfs1/gscratch/ssmc/USRS/PSI/Sukyong/kingcounty/3h/inputs/ssm_pnt_wq.dat
/mmfs1/gscratch/ssmc/USRS/PSI/Sukyong/kingcounty/3i/inputs/ssm_pnt_wq.dat
/mmfs1/gscratch/ssmc/USRS/PSI/Rachael/projects/KingCounty/SalishSeaModel/3j/hotstart/inputs/ssm_pnt_wq.dat
/mmfs1/gscratch/ssmc/USRS/PSI/Rachael/projects/KingCounty/SalishSeaModel/3k/hotstart/inputs/ssm_pnt_wq.dat
/mmfs1/gscratch/ssmc/USRS/PSI/Sukyong/kingcounty/3c/inputs/ssm_pnt_wq.dat
/mmfs1/gscratch/ssmc/USRS/PSI/Rachael/projects/KingCounty/SalishSeaModel/3l/hotstart

# Load 2014 loading values
This information was saved to file while creating input files in `create_scenario_pnt_wq_3k.py`

### Load loading inputs
This code comes from Ben Roberts.  I got the same results with my method but like Ben's better because it covers all variables. See [ssm_read_pnt_wq.ipynb](https://github.com/bedaro/ssm-analysis/blob/main/input_files/ssm_read_pnt_wq.ipynb)

From Ben Roberts:
```
The discharges are in m3/s, and cover a 24-hour period (technically the model linearly interpolates between them but that doesn't change the final result). The units for most constituents are mg/l and for the nitrogen constituents it's mg-N/l == g-N/m^3.

So when I load the NetCDF into a dataset named 'ds' and do this:
(ds['discharge'][:,0] * (ds['nh4'][:,0] + ds['no32'][:,0]) * 24 * 3600).sum()
```

In [15]:
inputs = {}
for run in runs:
    with open(ssm['paths']['nutrient_loading_inputs'][run]) as f:
        # The parsing logic here is is derived from the linkage instructions for the
        # model and direct examination of the source code

        # The first line does not contain important information and is treated only like
        # a filetype magic
        next(f)

        # The total number of discharge nodes
        num_qs = int(next(f))
        # All the node numbers with discharges
        #nodes = numpy.loadtxt([next(f) for l in range(num_qs)], comments='!', dtype=int)
        node_raw = StringIO('\n'.join([next(f) for l in range(num_qs)]))
        node_df = pandas.read_csv(node_raw, sep='\s+!\s+', names=('Node','Comment'),
                              dtype={'Node':numpy.int64,'Comment':object}, engine='python')
        node_df.set_index('Node', inplace=True)
        nodes = node_df.index.to_numpy()
        # Depth distribution fractions into each node. Skipping the first (node count) column
        vqdist = numpy.loadtxt([next(f) for l in range(num_qs)])[:,1:]

        num_times = int(next(f))

        # Initialize storage arrays
        times = numpy.zeros(num_times)
        qs = numpy.zeros((num_times, num_qs))
        # State variables in the order they are present in the file. These are also going
        # to be the NetCDF variable names
        statevars = ('discharge', 'temp', 'salt', 'tss',  'alg1', 'alg2', 'alg3', 'zoo1',
                                  'zoo2', 'ldoc', 'rdoc', 'lpoc', 'rpoc', 'nh4',  'no32',
                                  'urea', 'ldon', 'rdon', 'lpon', 'rpon', 'po4',  'ldop',
                                  'rdop', 'lpop', 'rpop', 'pip',  'cod',  'doxg', 'psi',
                                  'dsi',  'alg1p','alg2p','alg3p','dic',  'talk')
        inputs[run] = {}
        for v in statevars:
            inputs[run][v] = numpy.zeros((num_times, num_qs))

        for t in range(num_times):
            times[t] = float(next(f))
            for v in statevars:
                inputs[run][v][t,:] = numpy.loadtxt([next(f)])

In [4]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Creating dataframe with loading values
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
print('Creating WWTP and River loadings dataframe')
wwtp_load_df = pandas.DataFrame(wwtp_loads).sort_values(by=['wqm_baseline'], ascending=False)
river_load_df = pandas.DataFrame(river_loads).sort_values(by=['wqm_baseline'], ascending=False)
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Rename columns for report
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
wwtp_load_df=wwtp_load_df.rename(columns=ssm['run_information']['run_tag']['whidbey'])
river_load_df=river_load_df.rename(columns=ssm['run_information']['run_tag']['whidbey'])

Creating WWTP and River loadings dataframe


In [6]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Creating dataframe with discharge values
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
wwtp_flow_df = pandas.DataFrame({'Annual Total Flow (m^3/year)':total_flow['wqm_baseline'][case_wwtp]}, index=ssm['WWTP_names'])
river_flow_df = pandas.DataFrame({'Annual Total Flow (m^3/year)':total_flow['wqm_baseline'][case_river]}, index=ssm['river_names'])
# combine river discharge and loading in locations where there are duplicates (R_1, R_2)
flow = []
river_load = {}
# River discharge (not scenario-dependant)
for river in ssm['unique_river_names']:      
    flow.append(river_flow_df[river_flow_df.index.str.contains(river)]["Annual Total Flow (m^3/year)"].sum())
# River loading (scenario dependant)
for scenario in [*river_load_df]:
    river_load[scenario]=[]
    for river in ssm['unique_river_names']: 
        river_load[scenario].append(river_load_df[river_load_df.index.str.contains(river)][scenario].sum())
# dictionary of river discharge with one value per river instead of two (R_1 and R_2)
river_flow_df_corrected = pandas.DataFrame({'Annual Total Flow (m^3/year)':flow},index=ssm['unique_river_names'])
# dictionary of river loadings with one value per river instead of two (R_1 and R_2)
river_load_df_corrected = pandas.DataFrame(river_load,index=ssm['unique_river_names'])

In [13]:
river_flow_df_corrected

,Annual Total Flow (m^3/year)
Skagit,2.114061e+10
Snohomish,1.125372e+10
Stillaguamish,4.714366e+09
Whidbey east,1.050832e+08


In [21]:
# Combined information of loading and discharge for each river
river_load_df_corrected["Annual Total Flow (m^3/year)"] = river_flow_df_corrected["Annual Total Flow (m^3/year)"]
# Add column with WWTP dischard to the loading dataframe
wwtp_load_df["Annual Total Flow (m^3/year)"] = wwtp_flow_df["Annual Total Flow (m^3/year)"]
# [WWTP] Create dataframes for total loadings by (a) WWTP used in this study, and (b) all WWTPs
total_wwtp_loading_local_df = pandas.DataFrame(total_wwtp_loading_local, index=['Total WWTPs (altered in this report)'])
total_wwtp_loading_all_df = pandas.DataFrame(total_wwtp_loading_all, index=['Total WWTPs (all in model domain) '])
# [Rivers] Create dataframes for total loadings by (a) Rivers used in this study, and (b) all Rivers
total_river_loading_local_df = pandas.DataFrame(total_river_loading_local, index=['Total Rivers (altered in this report)'])
total_river_loading_all_df = pandas.DataFrame(total_river_loading_all, index=['Total Rivers (all in model domain) '])

In [22]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Combine dataframes with loadings added across WWTP/rivers used in this study vs. all in SSM
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Add the rows with total loading for local wwtps/rivers and all wwtps/rivers
wwtp_load_df=wwtp_load_df.append(total_wwtp_loading_local_df)
wwtp_load_df=wwtp_load_df.append(total_wwtp_loading_all_df)

river_load_df_corrected=river_load_df_corrected.append(total_river_loading_local_df)
river_load_df_corrected=river_load_df_corrected.append(total_river_loading_all_df)


/tmp/ipykernel_31218/3728283807.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  wwtp_load_df=wwtp_load_df.append(total_wwtp_loading_local_df)
/tmp/ipykernel_31218/3728283807.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  wwtp_load_df=wwtp_load_df.append(total_wwtp_loading_all_df)
/tmp/ipykernel_31218/3728283807.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  river_load_df_corrected=river_load_df_corrected.append(total_river_loading_local_df)
/tmp/ipykernel_31218/3728283807.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  river_load_df_corrected=river_load_df_corrected.append(total_river_loading_all_df)


In [24]:
river_load_df_corrected

,2014 Conditions,Reference,Wtp1,Wtp2,Wtp3,Wtp4,Wtp5,Wtp6,Wtp7,Wtp8,Wr1,Wr2,Wr3,Annual Total Flow (m^3/year)
Skagit,2.431596e+06,2.275192e+06,2.431596e+06,2.431596e+06,2.431596e+06,2.431596e+06,2.431596e+06,2.431596e+06,2.431596e+06,2.431596e+06,0.000000e+00,2.353434e+06,4.863186e+06,2.114061e+10
Snohomish,2.741295e+06,1.449164e+06,2.741295e+06,2.741295e+06,2.741295e+06,2.741295e+06,2.741295e+06,2.741295e+06,2.741295e+06,2.741295e+06,0.000000e+00,2.095213e+06,5.482590e+06,1.125372e+10
Stillaguamish,1.144373e+06,6.302542e+05,1.144373e+06,1.144373e+06,1.144373e+06,1.144373e+06,1.144373e+06,1.144373e+06,1.144373e+06,1.144373e+06,0.000000e+00,8.873084e+05,2.288745e+06,4.714366e+09
Whidbey east,7.186592e+04,1.461846e+04,7.186592e+04,7.186592e+04,7.186592e+04,7.186592e+04,7.186592e+04,7.186592e+04,7.186592e+04,7.186592e+04,0.000000e+00,4.324202e+04,1.437322e+05,1.050832e+08
Total Rivers (altered in this report),6.389129e+06,4.369229e+06,6.389129e+06,6.389129e+06,6.389129e+06,6.389129e+06,6.389129e+06,6.389129e+06,6.389129e+06,6.389129e+06,0.000000e+00,5.379198e+06,1.277825e+07,NaN
Total Rivers (all in model domain),2.551124e+07,1.999981e+07,2.551124e+07,2.551124e+07,2.551124e+07,2.551124e+07,2.551124e+07,2.551124e+07,2.551124e+07,2.551124e+07,1.912211e+07,2.450131e+07,3.190036e+07,NaN


In [25]:
# make README 
this_file = '=HYPERLINK("https://github.com/UWModeling/KingCounty-Rachael/blob/main/notebooks/reports/Table1_NutrientLoadings.ipynb")'
run_description = '=HYPERLINK("https://github.com/RachaelDMueller/KingCounty-Rachael/blob/main/docs/supporting/KingCounty_Model_Runs.xlsx","KingCounty_Model_Runs.xlsx (USING ORIGINAL RUN TAGS!!!)")'

created_by = 'Rachael D. Mueller'
created_at = 'Puget Sound Institute'
created_from = 'Model results produced by Su Kyong Yun at the Salish Sea Modeling Center'
units='kg/year'
created_on = date.today().strftime("%B %d, %Y")
contact = 'Rachael D Mueller (rdmseas@uw.edu)'
header = {
    ' ':[created_by, created_at, created_on, this_file, 
        units, contact, created_from, 
        run_description]
}
header_df = pandas.DataFrame(header, index=[
    'Created by',
    'Created at',                           
    'Created on',
    'Created with',
    'Units',
    'Contacts',
    'Modeling by',
    'Model Run Overview'])

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Save output to excel
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
excel_output_path = pathlib.Path('/mmfs1/gscratch/ssmc/USRS/PSI/Rachael/projects/KingCounty/reports')
print('*************************************************************')
print('Writing spreadsheet to: ',excel_output_path)
print('*************************************************************')
if os.path.exists(excel_output_path)==False:
    print(f'creating: {excel_output_path}')
    os.umask(0) #clears permissions
    os.makedirs(excel_output_path, mode=0o777,exist_ok=True)
with pandas.ExcelWriter(
    excel_output_path/f'Table1_NutrientLoadings_from_input_{case}.xlsx', mode='w') as writer:  
    wwtp_load_df.to_excel(writer, sheet_name=f'WWTP ({case})')
    river_load_df_corrected.to_excel(writer, sheet_name=f'Rivers ({case})')
    header_df.to_excel(writer, sheet_name='README')
print(f'Number of WWTPs in this case: {len(total_scenario_wwtp_nitrogen_df)}')
print(f'Number of rivers in this case: {len(total_scenario_river_nitrogen_df)}')
print(f'Number of WWTPs in model: {len(total_wwtp_nitrogen_df)}')
print(f'Number of rivers in model: {len(total_river_nitrogen_df)}')


*************************************************************
Writing spreadsheet to:  /mmfs1/gscratch/ssmc/USRS/PSI/Rachael/projects/KingCounty/reports
*************************************************************
Number of WWTPs in this case: 17
Number of rivers in this case: 7
Number of WWTPs in model: 99
Number of rivers in model: 160


# Test script

In [3]:
case='whidbey'
# open yaml configuration file created by notebook `SSM_config_{case}.ipynb` 
with open(f'../../etc/SSM_config_{case}.yaml', 'r') as file:
    ssm = yaml.safe_load(file)
# load list of directory paths for SSM nutrient loading .dat files
runs = [*ssm['paths']['nutrient_loading_inputs']]
# print out file list
for run in runs:
    print(ssm['paths']['nutrient_loading_inputs'][run])
# # establish directory path for ssm_pnt_wq_station_info.xlsx
# in_dir = pathlib.Path('/mmfs1/gscratch/ssmc/USRS/PSI/Rachael/projects/KingCounty/spreadsheets')

### START: move to util script ###

# Adaptation of Ben's method for processing output    
inputs = {}
for run in runs:
    with open(ssm['paths']['nutrient_loading_inputs'][run]) as f:
        # The parsing logic here is is derived from the linkage instructions for the
        # model and direct examination of the source code

        # The first line does not contain important information and is treated only like
        # a filetype magic
        next(f)

        # The total number of discharge nodes
        num_qs = int(next(f))
        # All the node numbers with discharges
        #nodes = np.loadtxt([next(f) for l in range(num_qs)], comments='!', dtype=int)
        node_raw = StringIO('\n'.join([next(f) for l in range(num_qs)]))
        node_df = pandas.read_csv(node_raw, sep='\s+!\s+', names=('Node','Comment'),
                              dtype={'Node':numpy.int64,'Comment':object}, engine='python')
        node_df.set_index('Node', inplace=True)
        nodes = node_df.index.to_numpy()
        # Depth distribution fractions into each node. Skipping the first (node count) column
        vqdist = numpy.loadtxt([next(f) for l in range(num_qs)])[:,1:]

        num_times = int(next(f))

        # Initialize storage arrays
        times = numpy.zeros(num_times)
        qs = numpy.zeros((num_times, num_qs))
        # State variables in the order they are present in the file. These are also going
        # to be the NetCDF variable names
        statevars = ('discharge', 'temp', 'salt', 'tss',  'alg1', 'alg2', 'alg3', 'zoo1',
                                  'zoo2', 'ldoc', 'rdoc', 'lpoc', 'rpoc', 'nh4',  'no32',
                                  'urea', 'ldon', 'rdon', 'lpon', 'rpon', 'po4',  'ldop',
                                  'rdop', 'lpop', 'rpop', 'pip',  'cod',  'doxg', 'psi',
                                  'dsi',  'alg1p','alg2p','alg3p','dic',  'talk')
        inputs[run] = {}
        for v in statevars:
            inputs[run][v] = numpy.zeros((num_times, num_qs))

        for t in range(num_times):
            times[t] = float(next(f))
            for v in statevars:
                inputs[run][v][t,:] = numpy.loadtxt([next(f)])

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Create dictionaries of loading and discharge information by run
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
loading={}
total_flow={}
total_nitrogen={}
#total_annual_nitrogen={}
for run in runs:
    print(run)
    loading[run] = (inputs[run]['discharge'] * (inputs[run]['nh4'] + inputs[run]['no32']) * 24 * 3600)/1000 #m3/s*mg/l -> kg/day
    total_flow[run] = (inputs[run]['discharge'] * 24 * 3600).sum(axis=0) # annual discharge m3/year
    total_nitrogen[run]=loading[run].sum(axis=0) # annual loading over 366 days (g/year)
    #total_annual_nitrogen[run] = total_nitrogen[run].sum() # g/year -> kg/year

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# import source location names from excel
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# information in excel spreadsheet: type (" River" or " Point Source"), node_id, layer_distribute, basin, country, NH4[kg/year], NO3NO2[kg/year], N-load
source_locations = pandas.read_excel(in_dir/'ssm_pnt_wq_station_info.xlsx',index_col='Unnamed: 0')
river_locations = source_locations[source_locations['type']==' River']
wwtp_locations = source_locations[source_locations['type']==' Point Source']
# make lists of names for: 
all_rivers_list = river_locations.index.to_list() # river input locations
all_wwtps_list = wwtp_locations.index.to_list()   # WWTP input locations
all_locations_list = source_locations.index.to_list()  # both river and WWTP input locations
all_locations_type_list = source_locations['type'].to_list() # type of input (" River" or " Point Source") to eliminate duplicate names

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Create dataframes of information using names from excel as indices
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# NOTE: Some names are redundant for rivers and WWTP so we need to use both name and type (" River" vs. " Point Source")
total_flow_df  = pandas.DataFrame(
    total_flow, 
    index=all_locations_list,
    columns=ssm['run_information']['run_tag']['whidbey'] # Assign report names to columns
)
total_loading_df = pandas.DataFrame(
    total_nitrogen, 
    index=all_locations_list,
    columns=ssm['run_information']['run_tag']['whidbey'] # Assign report names to columns
)
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Use both name of source and type to create lists of wwtp and river sources
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Create boolean arrays with True for only WWTP and River sources used in this cases 
case_wwtp = numpy.asarray(total_loading_df.index.isin(ssm['WWTP_names']) & (source_locations['type']==' Point Source'))
case_river = numpy.asarray(total_loading_df.index.isin(ssm['river_names']) & (source_locations['type']==' River'))
# Create boolean arrays with True for all WWTP and River sources used in this study
all_wwtp= numpy.asarray(total_loading_df.index.isin(all_wwtps_list) & (source_locations['type']==' Point Source'))
all_river = numpy.asarray(total_loading_df.index.isin(all_rivers_list) & (source_locations['type']==' River'))
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Calculate the total nitrogen loading for rivers and wwtps
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# modified in a particular case
total_scenario_wwtp_nitrogen_df = total_loading_df[case_wwtp]
total_scenario_river_nitrogen_df = total_loading_df[case_river]
# included in SSM
total_wwtp_nitrogen_df = total_loading_df[all_wwtp]
total_river_nitrogen_df = total_loading_df[all_river]
# total nitrogen loads, by source and scenario
wwtp_loads = total_loading_df[case_wwtp]
river_loads = total_loading_df[case_river]
# total nitrogen loads across local and all sources, by scenario
# rename columns in the process
total_wwtp_loading_local={}
total_river_loading_local={}
total_wwtp_loading_all={}
total_river_loading_all={}
for scenario in [*total_scenario_wwtp_nitrogen_df]:
    total_wwtp_loading_local[ssm['run_information']['run_tag']['whidbey'][scenario]]=total_scenario_wwtp_nitrogen_df[scenario].sum()
    total_river_loading_local[ssm['run_information']['run_tag']['whidbey'][scenario]]=total_scenario_river_nitrogen_df[scenario].sum()
    total_wwtp_loading_all[ssm['run_information']['run_tag']['whidbey'][scenario]]=total_wwtp_nitrogen_df[scenario].sum()
    total_river_loading_all[ssm['run_information']['run_tag']['whidbey'][scenario]]=total_river_nitrogen_df[scenario].sum()

/mmfs1/gscratch/ssmc/USRS/PSI/Sukyong/kingcounty/WQM/WQM/hotstart/inputs/ssm_pnt_wq.dat
/mmfs1/gscratch/ssmc/USRS/PSI/Sukyong/kingcounty/WQM_REF/WQM_REF/hotstart/inputs/ssm_pnt_wq.dat
/mmfs1/gscratch/ssmc/USRS/PSI/Sukyong/kingcounty/3b/inputs/ssm_pnt_wq.dat
/mmfs1/gscratch/ssmc/USRS/PSI/Sukyong/kingcounty/3e/inputs/ssm_pnt_wq.dat
/mmfs1/gscratch/ssmc/USRS/PSI/Sukyong/kingcounty/3f/inputs/ssm_pnt_wq.dat
/mmfs1/gscratch/ssmc/USRS/PSI/Sukyong/kingcounty/3g/inputs/ssm_pnt_wq.dat
/mmfs1/gscratch/ssmc/USRS/PSI/Sukyong/kingcounty/3h/inputs/ssm_pnt_wq.dat
/mmfs1/gscratch/ssmc/USRS/PSI/Sukyong/kingcounty/3i/inputs/ssm_pnt_wq.dat
/mmfs1/gscratch/ssmc/USRS/PSI/Rachael/projects/KingCounty/SalishSeaModel/3j/hotstart/inputs/ssm_pnt_wq.dat
/mmfs1/gscratch/ssmc/USRS/PSI/Rachael/projects/KingCounty/SalishSeaModel/3k/hotstart/inputs/ssm_pnt_wq.dat
/mmfs1/gscratch/ssmc/USRS/PSI/Sukyong/kingcounty/3c/inputs/ssm_pnt_wq.dat
/mmfs1/gscratch/ssmc/USRS/PSI/Rachael/projects/KingCounty/SalishSeaModel/3l/hotstart

In [6]:
wwtp_load_df

,2014 Conditions,Reference,Wtp1,Wtp2,Wtp3,Wtp4,Wtp5,Wtp6,Wtp7,Wtp8,Wr1,Wr2,Wr3,Annual Total Flow (m^3/year)
OF100,3.711855e+05,1.748920e+03,0.000000e+00,3.711855e+05,3.711855e+05,0.000000e+00,3.711855e+05,0.000000e+00,7.357751e+05,4.587489e+05,3.711855e+05,3.711855e+05,3.711855e+05,1.571218e+07
Everett Snohomish,3.645986e+05,1.172538e+03,0.000000e+00,3.645986e+05,3.645986e+05,0.000000e+00,0.000000e+00,3.645986e+05,0.000000e+00,2.747435e+05,3.645986e+05,3.645986e+05,3.645986e+05,1.625816e+07
Mt Vernon,1.339718e+05,7.156587e+02,0.000000e+00,1.339718e+05,0.000000e+00,1.339718e+05,1.339718e+05,1.339718e+05,1.339718e+05,1.339718e+05,1.339718e+05,1.339718e+05,1.339718e+05,5.658301e+06
Marysville,1.327813e+05,2.795250e+02,0.000000e+00,1.327813e+05,0.000000e+00,1.327813e+05,1.327813e+05,1.327813e+05,1.327813e+05,1.327813e+05,1.327813e+05,1.327813e+05,1.327813e+05,4.330835e+06
Oak Harbor Lagoon,7.605730e+04,2.848760e+02,0.000000e+00,7.605730e+04,0.000000e+00,7.605730e+04,7.605730e+04,7.605730e+04,7.605730e+04,7.605730e+04,7.605730e+04,7.605730e+04,7.605730e+04,2.518107e+06
Lake Stevens 002,6.390890e+04,4.449715e+02,0.000000e+00,6.390890e+04,0.000000e+00,6.390890e+04,6.390890e+04,6.390890e+04,6.390890e+04,6.390890e+04,6.390890e+04,6.390890e+04,6.390890e+04,3.614861e+06
Snohomish,3.211112e+04,2.648870e+02,0.000000e+00,3.211112e+04,0.000000e+00,3.211112e+04,3.211112e+04,3.211112e+04,3.211112e+04,3.211112e+04,3.211112e+04,3.211112e+04,3.211112e+04,1.963483e+06
La Conner,1.141406e+04,5.101155e+01,0.000000e+00,0.000000e+00,1.141406e+04,1.141406e+04,1.141406e+04,1.141406e+04,1.141406e+04,1.141406e+04,1.141406e+04,1.141406e+04,1.141406e+04,4.166389e+05
Mukilteo,9.015299e+03,1.992815e+02,0.000000e+00,0.000000e+00,9.015299e+03,9.015299e+03,9.015299e+03,9.015299e+03,9.015299e+03,9.015299e+03,9.015299e+03,9.015299e+03,9.015299e+03,1.620216e+06
Stanwood,6.313198e+03,9.850011e+01,0.000000e+00,0.000000e+00,6.313198e+03,6.313198e+03,6.313198e+03,6.313198e+03,6.313198e+03,6.313198e+03,6.313198e+03,6.313198e+03,6.313198e+03,7.921765e+05
